# BERT for Cloze problem in python 2
Since the source codes provided for ScratchGan are in python 2 and are used with Tensorflow 1.x, let's try follow that style

## 1. Loading and Initializing

### 1.1 Sentence processing functions

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
print("Using Tensorflow version: " + tf.__version__)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()[-1])

BERT_DIR = "/home/aufish/Downloads/bert"
# tf.enable_eager_execution()

Using Tensorflow version: 1.14.0
name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6967551591
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8153560289178769018
physical_device_desc: "device: 0, name: GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5"



In [2]:
bert_module = hub.KerasLayer(BERT_DIR, trainable=True)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
# tokenizer
from bert import tokenization

def create_tokenizer(vocab_file, do_lower_case=False):
    return tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer(BERT_DIR + "/assets/vocab.txt")

def convert_sentence_to_features(sentence, tokenizer, max_seq_len=50):
    tokens = ['[CLS]']
    tokens.extend(tokenizer.tokenize(sentence))
    if len(tokens) > max_seq_len-1:
        tokens = tokens[:max_seq_len-1]
    tokens.append('[SEP]')
    
    segment_ids = [0] * len(tokens)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)

    #Zero Mask till seq_length
    zero_mask = [0] * (max_seq_len-len(tokens))
    input_ids.extend(zero_mask)
    input_mask.extend(zero_mask)
    segment_ids.extend(zero_mask)
    
    return input_ids, input_mask, segment_ids

def convert_sentences_to_features(sentences, tokenizer, max_seq_len=50):
    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []
    
    for sentence in sentences:
        input_ids, input_mask, segment_ids = convert_sentence_to_features(sentence, tokenizer, max_seq_len)
        all_input_ids.append(input_ids)
        all_input_mask.append(input_mask)
        all_segment_ids.append(segment_ids)
    
    return all_input_ids, all_input_mask, all_segment_ids

import random, copy
import numpy as np
def make_rand_mask(input_ids, input_mask, vocab_size, segment_id_vals=None):
    ''' 
    Only make mask for one sentence
    input_ids: the ids of words in the sentences
    input_mask: initial mask (1 if there is a word; 0 for padding)
    returns
    input_mask: replace one bit of 1 with 0, meaning that the word will be masked
    mask_word_ids: the id of words that are masked
    pure_ids: ids in number instead of one-hot (to generate weights per masked word)
    segment_id_vals: mark the masked word with segment id 1
    sequential_output_filter: the masked word index
    '''
    
    new_input_mask = copy.deepcopy(input_mask)
#     mask_word_ids = np.zeros(vocab_size)
    pure_ids = []

    total_word = sum(input_mask)
    mask_word = random.randint(0, total_word-1)

    pure_ids.append(input_ids[mask_word])
    assert new_input_mask[mask_word] == 1
    new_input_mask[mask_word] = 0
#     mask_word_ids[input_ids[mask_word]] = 1.0
                
    return new_input_mask, segment_id_vals, mask_word

### 1.2 Blank filler model

In [13]:
class WordPredictor(tf.keras.Model):
    # The output means, how possible the given word may fit into the blank
    def __init__(self, class_num, bert=bert_module, dropout=0.1):
        super(WordPredictor, self).__init__()
        self.bert = bert
        self.drop = tf.keras.layers.Dropout(rate=dropout, trainable=True)
        
        self.dense = tf.keras.layers.Dense(
            class_num,
            activation=None,
            kernel_initializer='glorot_uniform',
            name='word_prediction',
            trainable=True)
        
    def call(self, inputs, mask_loc):
        # When passed in, all tensors are stacked in one, split it into a list
        inputs = tf.unstack(tf.cast(inputs, tf.dtypes.int32), axis=1)

        pooled, sequential = self.bert(inputs)
        
        # select one from each batch
        s = tf.gather_nd(sequential, [(i, mask_loc[i]) for i in range(sequential.shape[0])])
        # s now has shape (batch_size * 768)
        
        x = self.drop(s)
        return self.dense(x)

### 1.2.1 Sanity test

In [5]:
model = WordPredictor(1)

opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(opt)

In [12]:
model.load_weights("./word_predictor_9")

## 2. Prepare data

In [5]:
import numpy as np
# extracted from emnlp
DATA_FILE = "./sentences.txt"

MASK_ID = tokenizer.convert_tokens_to_ids(['[MASK]'])[0]

def parse_line(line):    
    input_ids, input_mask, segment_ids = convert_sentence_to_features(line, tokenizer, max_seq_len=30)
        
    input_mask, segment_ids, mask_position = \
        make_rand_mask(input_ids, input_mask, len(tokenizer.vocab), segment_ids)
    
    label = input_ids[mask_position]
    input_ids[mask_position] = MASK_ID
    
    return [input_ids, input_mask, segment_ids], [mask_position, label]

def create_dataset(filename = DATA_FILE, data_size = 268528, batch_size = 10):
    dataset = tf.data.TextLineDataset([filename])
    
    dataset = dataset.map(lambda x : tf.numpy_function(parse_line, [x], [tf.int64, tf.int64]))
    
    # dataset = dataset.shuffle(data_size, reshuffle_each_iteration=True)
    
    dataset = dataset.batch(batch_size)
    
    return dataset

In [6]:
tokenizer.convert_tokens_to_ids(['[MASK]'])

[103]

### 2.1 Model calling sanity test

In [8]:
temp_batch_size = 3
dataset = create_dataset(batch_size = temp_batch_size)

In [11]:
# Processing data without model
for (bert_input, label) in dataset.take(1):
    bert_input = tf.reshape(bert_input, [temp_batch_size, 3, 30])
    print(type(bert_input))
    print(bert_input)
    print(label)
    inputs = tf.unstack(tf.cast(bert_input, tf.dtypes.int32), axis=1)
    
#     pooled, sequential = bert_module(inputs)
    
#     print(sequential.shape)
    
# #     seq_output_filter = tf.cast(seq_output_filter, tf.dtypes.float32)
# #     r = tf.tensordot(seq_output_filter,  sequential, axes=(1, 1))
    
#     mask, label = tf.unstack(label, axis=1)
#     r = tf.gather_nd(sequential, [(i, mask[i]) for i in range(temp_batch_size)])
    
#     print(r)
#     print(label)

InvalidArgumentError: exceptions.TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'
Traceback (most recent call last):

  File "/home/aufish/.local/lib/python2.7/site-packages/tensorflow/python/ops/script_ops.py", line 209, in __call__
    ret = func(*args)

����
����
TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'


	 [[{{node PyFunc}}]] [Op:IteratorGetNextSync]

## 4. Predict and usage

In [105]:
from tensorflow.python.keras.backend import set_session
graph = tf.get_default_graph()
sess = tf.Session()
set_session(sess)

In [106]:
MASK_ID = tokenizer.convert_tokens_to_ids(['[MASK]'])[0]
def blank_word_predict(model, sentence, blank_loc, candidate_num=10):
    # Given a sentence and at which location (0-indexed) it is blank
    # return the predicted word
    ids, masks, seg_ids = convert_sentence_to_features(sentence, tokenizer)
    
    # adjust input_mask, reset the randomly selected mask and set with blank_loc
    masks[blank_loc] = 0
    
    ids[blank_loc] = MASK_ID
    
    bert_input = tf.stack([ids, masks, seg_ids])
    bert_input = tf.reshape(bert_input, (1, bert_input.shape[0], bert_input.shape[1]))
        
    
    output = model(bert_input, [blank_loc])
    with graph.as_default():
        set_session(sess)
        output = output.eval(session=sess)
#         print(output.eval(session=sess))
    
#     sess.run(output)
        
    print(type(output))
    return tf.argsort(output, direction='DESCENDING')[0, :candidate_num]

In [107]:
model = WordPredictor(len(tokenizer.vocab))
model.load_weights("./word_predictor_9")

In [108]:
sentence = "the blank has caused panic around the world"

result = blank_word_predict(model, sentence, 2, 5)

print(sess._closed)

<type 'numpy.ndarray'>
False


In [109]:
print(type(result))
print(result)

<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("strided_slice_11:0", shape=(5,), dtype=int32)


In [102]:
with tf.Session() as sess:
    result = result.eval()
    print(result)

for i in range(result.shape[0]):
    print(tokenizer.convert_ids_to_tokens([result[i]]))

[4497 2035 1692 4598 7942]
[u'incident']
[u'attack']
[u'case']
[u'shooting']
[u'virus']


In [104]:
print(sess._closed)

True
